In [1]:
from src.ssvm import optimizer, sampler, tracker, pattern
import numpy as np
import time, pickle, sys, os
from src.ssvm.utils import load_dataset
from src.utils import Region, get_loss
import warnings
warnings.filterwarnings("ignore")

In [2]:
data_names = [tmp for tmp in os.listdir('./data/TinyTLP/') if not tmp.endswith('zip')]
data_names[:5], len(data_names)

(['Alladin', 'Aquarium1', 'Aquarium2', 'Badminton1', 'Badminton2'], 50)

In [3]:
data_path = './data/TinyTLP/{}'.format(data_names[0])

In [15]:
frames, truths = load_dataset(data_path, mode='TLP')
#regions = [Region(list(truth), data_mode='tlp', region_mode='raw') for truth in truths]

In [14]:
truths[0], regions[0].get_label()

(array([520, 401,  94, 279]), (401, 427, 279, 279))

In [11]:
# Initialize tracker
config = {
    "rescale": 0.2,
    "search": 1.3,
    "step": 2,
    "P": 5, "Q": 10,
    "sv_max": 100,
}

In [7]:
# Initialize tracker
config = {
    "rescale": 0.2,
    #"search": 1.3,
    "search": 0.1,
    "step": 2,
    "P": 5, "Q": 10,
    "sv_max": 20,
}

In [4]:
def ssvm_object_tracking(frames, truths):
    target = np.array(truths[0])
    ssvm = tracker.Tracker(frames, target, config)
    losses = []
    for (frame, output), truth in zip(ssvm.track(), truths):
        loss = get_loss(Region(output, data_mode='tlp', region_mode='raw'), Region(truth, data_mode='tlp', region_mode='raw'))
        losses.append(loss)
    return losses

In [5]:
data_names[33]

'Jet5'

In [16]:
losses_all = []
with open('ssvm.log', 'a') as f:
    for i, video_name in enumerate(data_names):
        data_path = './data/TinyTLP/{}'.format(video_name)
        frames, truths = load_dataset(data_path, mode='TLP')
        losses = ssvm_object_tracking(frames, truths)
        losses_all.append(losses)
        print(video_name, np.mean(losses))
        to_log = [video_name] + [str(np.mean(losses))] + [str(loss) for loss in losses]
        f.write('\t'.join(to_log) + '\n')

KinBall1 16.893661874456967
KinBall2 6.374371492238126
KinBall3 70.26862037240099
Lion 143.32968873918287
Mohiniyattam 225.12214613939776
MotorcycleChase 43.72832085382151
Parakeet 149.84465958727802
PolarBear1 76.28920627911646
PolarBear2 335.04106394361673
PolarBear3 185.0239052588594
Puppies1 22.72198415850151
Puppies2 198.13765134045957
Rope 13.516999031026838
Sam 260.59710323479743
Violinist 33.50760024387366
ZebraFish 157.41174424698985


In [41]:
truths[0]

array([ 123,  261, 1047,  334])